# Dual-Body Dissipation
Joe P. Renaud<sup>1,2,3</sup>, Wade G. Henning<sup>4,2,3</sup>, Prabal Saxena<sup>4,2,3</sup>, Marc Neveu<sup>4,2</sup>, Amirhossein Bagheri<sup>5</sup>, Avi Mandell<sup>2,3</sup>, Terry Hurford<sup>2</sup>

<sub><sup>1: Universities Space Research Association</sub></sup>
<sub><sup>2: NASA Goddard Space Flight Center</sub></sup>
<sub><sup>3: Sellers Exoplanet Environments Collaboration</sub></sup>
<sub><sup>4: University of Maryland, College Park</sub></sup>
<sub><sup>5: Institute of Geophysics, ETH Z¨urich, Switzerland</sub></sup>

This notebook contains figures and calculations related to the manuscript: [Tidal Dissipation in Dual-Body, Highly Eccentric, and Non-synchronously Rotating Systems: Applications to Pluto-Charon and the Exoplanet TRAPPIST-1e](https://ui.adsabs.harvard.edu/abs/2020arXiv201011801R/abstract).

*Some values may be different from the ones used in the original manuscript so figures may not match exactly. All models and equations are, however, identical.*

# Pre-Analysis Setup

In [1]:
analysis_version = '3.0'
analysis_version_save = analysis_version

import os
import math
import time
import warnings
from typing import List, Tuple

# Third Party Package Import
import numpy as np
from scipy.constants import G
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset, inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable

from ipywidgets import interact
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

# NaNs will pop up in some analysis. Ignore those warnings
_ = np.seterr(invalid='ignore')

%matplotlib ipympl

In [2]:
# TidalPy Import
from TidalPy.utilities.types import NoneType, FloatArray
from TidalPy.toolbox.conversions import semi_a2orbital_motion, orbital_motion2semi_a, rads2days, days2rads
from TidalPy.utilities.numpy_helper.array_other import neg_array_for_log_plot, find_nearest
from TidalPy.utilities.graphics.cmaps import vik_map, tofino_map, lajolla_map, lajolla_map_r, cork_map
from TidalPy.constants import radius_earth, mass_earth, mass_solar, radius_solar, au 
from TidalPy.toolbox import quick_tidal_dissipation

# combine them and build a new colormap
_colors = np.vstack((lajolla_map_r(np.linspace(0., 1, 128)), lajolla_map(np.linspace(0., 1, 128))))
divergence_lajolla = mcolors.LinearSegmentedColormap.from_list('divergence_lajolla', _colors)

### Analysis Input Area

In [3]:
auto_save_plots_ck = \
    widgets.Checkbox(value=False, description='Auto Save Plots')

resolution_text = widgets.Label(value='Calculation Resolution:')
resolution_selection = \
    widgets.RadioButtons(
        options=['LowRes-Interactive Mode', 'MedRes', 'HighRes', 'HighRes - Publication Mode - Warning Can Lead to Memory Issues!'],
        value='MedRes',
        description='',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%', height='80px'))

input_area = widgets.VBox([auto_save_plots_ck, resolution_text, resolution_selection])
input_area

### Define Plot Styles

In [4]:
plt.rcParams.update({'font.size': 14})

satur_perc = .8
vik_N = vik_map.N

# Colorblind safe reds and blues
red = vik_map(math.floor(vik_N * satur_perc))
blue = vik_map(math.floor(vik_N * (1 - satur_perc)))

## Define Constant Parameters and Properties
We will not use the planetary properties pre-defined in TidalPy to ensure that the paper's figures can be reproduced in the future if any changes are made the pre-defined TidalPy values. For the same reason, we will not be using TidalPy's OOP or build_world methods.

### Material Properties & Complex Compliance Functions

In [5]:
rock_viscosity = 1.0e22
rock_shear = 5.0e10
ice_viscosity = 1.0e14
ice_shear = 1.e9 # 3.3e9

# Store line color and complex compliance inputs for each rheology.
rheologies = {
    'Andrade': ('b', 'andrade', (0.3, 1.0)),
    'Maxwell': ('k', 'maxwell', tuple()),
    'Sundberg': ('m', 'sundberg', (0.2, 0.02, 0.3, 1.0)),
    'Sundberg-Cooper': ('m', 'sundberg', (0.2, 0.02, 0.3, 1.0)),
    'CPL': ('g', 'cpl', tuple()),
    'CTL': ('c', 'ctl', tuple()),
    'Off': ('gray', 'Off', tuple())
}

### Planetary Properties

In [6]:
objects = dict()

pluto = {
    'name': 'Pluto',
    'type': 'ice',
    'mass': 0.01328e24,
    'radius': 1.1883e6,
    'semi_a': 19.596e6,  # Charon's Semi-Major Axis
    'spin_rate': 2. * np.pi / (86400. * 6.387229),
    'tidal_scale': .1,
    'fixed_q': 100.,
    'fixed_dt': (1. / 100.) * (2. * np.pi / (86400. * 6.387229))**(-1),
    'k2': None,
    'host': None
}
objects['Pluto'] = pluto

charon = {
    'name': 'Charon',
    'type': 'ice',
    'mass': 0.001603e24,
    'radius': 0.606e6,
    'semi_a': 19.596e6,
    'spin_rate': 2. * np.pi / (86400. * 6.387229),
    'tidal_scale': .1,
    'fixed_q': 100.,
    'fixed_dt': (1. / 100.) * (2. * np.pi / (86400. * 6.387229))**(-1),
    'k2': None,
    'host': pluto
}
objects['Charon'] = charon
pluto['host'] = charon

TRAPPIST1 = {
    'name': 'TRAPPIST-1',
    'type': 'star',
    'mass': 1.6e29,
    'radius': 81.4e6,
    'tidal_scale': 1.,
    'fixed_q': 8000.,
    'fixed_dt': (1. / 8000.) * (2. * np.pi / (86400. * 6.099))**(-1),
    'k2': 0.33,
    'host': None
}
objects['TRAPPIST-1'] = TRAPPIST1

TRAPPIST1_e = {
    'name': 'TRAPPIST1-e',
    'type': 'rock',
    'mass': 4.6e24,      # Nearly 94% error in this measurement
    'radius': 5.995e6, # About a 5% error in this measurement
    'semi_a': 4.3807e9,         # 3% error
    'tidal_scale': 1.,
    'fixed_q': 900.,
    'fixed_dt': (1. / 900.) * (2. * np.pi / (86400. * 6.099))**(-1),
    'k2': None,
    'host': TRAPPIST1
}
objects['TRAPPIST-1 e'] = TRAPPIST1_e

# Proxima data from Suarez Mascareno 2020
Proxima = {
    'name': 'Proxima',
    'type': 'star',
    'mass': 0.1221 * mass_solar, # Mann+2015 via SM2020
    'radius': 0.141 * radius_solar,
    'tidal_scale': 1.,
    'fixed_q': 100.,
    'fixed_dt': (1. / 8000.) * (2. * np.pi / (86400. * 11.186))**(-1),
    'k2': 0.33,
    'host': None
}
objects['Proxima'] = Proxima

Proxima_b = {
    'name': 'Proxima',
    'type': 'rock',
    'mass': 1.25 * mass_earth, # Mann+2015 via SM2020 GP+2 column
    'radius': 1.1 * radius_earth, # Total guess. Should be treated as a semi-free parameter based on mass.
    'semi_a': orbital_motion2semi_a(2. * np.pi / (86400. * 5.152), 0.1221 * mass_solar, 1.25 * mass_earth),
    'tidal_scale': 1.,
    'fixed_q': 100.,
    'fixed_dt': (1. / 100.) * (2. * np.pi / (86400. * 11.186))**(-1),
    'k2': None,
    'host': Proxima
}
objects['Proxima b'] = Proxima_b

for object_name, object_data in objects.items():

    # Calculate basic geometry
    object_data['volume'] = (4. / 3.) * np.pi * object_data['radius']**3
    object_data['density_bulk'] = object_data['mass'] / object_data['volume']
    object_data['gravity_surface'] = G * object_data['mass'] / object_data['radius']**2
    object_data['beta'] = object_data['radius'] * object_data['gravity_surface'] * object_data['density_bulk']
    object_data['moi'] = (2. / 5.) * object_data['mass'] * object_data['radius']**2
    if 'moi_scale' in object_data:
        object_data['moi'] *= object_data['moi_scale']
    
    if object_data['type'] == 'ice':
        object_data['k2'] = (3. / 19.) * (object_data['beta'] / ice_shear)
    elif object_data['type'] == 'rock':
        object_data['k2'] = (3. / 19.) * (object_data['beta'] / rock_shear)

    # If satellite, calculate orbital frequency
    if object_data['host'] is not None:
        host = object_data['host']
        object_data['orbital_freq'] = semi_a2orbital_motion(object_data['semi_a'], host['mass'], object_data['mass'])

## Load Tidal Calculation Functions


In [7]:
from TidalPy.toolbox.quick_tides import (dual_dissipation_from_dict_or_world_instance,
    single_dissipation_from_dict_or_world_instance)

In [8]:
def get_resolution():
    
    if resolution_selection.value == 'LowRes-Interactive Mode':
        one_d_res = 5000
    elif resolution_selection.value == 'MedRes':
        one_d_res = 50000
    elif resolution_selection.value == 'HighRes':
        one_d_res = 350000
    elif resolution_selection.value == 'HighRes - Publication Mode - Warning Can Lead to Memory Issues!':
        one_d_res = 722500
    else:
        one_d_res = 10
    
    two_d_res = int(math.floor(one_d_res**(1/2)))
    
    return one_d_res, two_d_res

# Main Analysis & Plotting

## Rheological Method Compated to CPL and CTL

In [9]:
def cpl_ctl_compare(target_object, host_object, obliquity: float = 0., eccentricity: float = 0.1,
                    target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    use_nsr = True, tidal_order_l: int = 2, eccentricity_truncation: int = 6,
                    add_save_name: str = None,
                    save_fig=auto_save_plots_ck.value):
    
    # Setup Domain
    oned_n, twod_n = get_resolution()
    orbital_motion = target_object['orbital_freq']
    spin_scale_domain = np.linspace(0.5, 10, oned_n)
    spin_scale_domain_res = np.asarray([0.5, 1., 1.5, 2., 2.5, 3., 3.5, 4., 4.5, 5., 5.5, 6.])
    spin_scale_domain = np.concatenate((spin_scale_domain, spin_scale_domain_res))
    spin_scale_domain = np.sort(spin_scale_domain)
    spin_frequency = spin_scale_domain * orbital_motion
    spin_period = rads2days(spin_frequency)
    if use_nsr:
        # Contant orbital motion at its original value. But, make it an array for calculations
        orbital_freq = orbital_motion * np.ones_like(spin_frequency) 
    else:
        # Set the orbital motion equal to the spin-rate
        orbital_freq = spin_frequency
    
    # Setup Figure
    fig, axes = plt.subplots(ncols=2, figsize=(1.5 * 6.4, 4.8))
    fig.subplots_adjust(wspace=0.4)
    heating_ax = axes[0]
    eccen_ax = axes[1]
    heating_ax.set(xlabel='Spin Period [day]', ylabel='Tidal Heating [W]', yscale='log')
    eccen_ax.set(xlabel='Spin Period [day]', ylabel='$\\dot{e}$ [yr$^{-1}$]', yscale='log')
    if use_nsr:
        fig.suptitle(f'Orbital Period = {rads2days(orbital_motion):.1f} Days', fontsize=12)
    else:
        fig.suptitle('Orbital Period = Spin Period', fontsize=12)

    # Build rheology models
    rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]
    cases = {
        'CTL': ('--', 'ctl', tuple()),
        'CPL': (':', 'cpl', tuple()),
        target_rheo: ('-', rheo_model_name, rheo_input)
    }
    
    for rheo_nice_name, (line_style, rheo_name_, rheo_input_) in cases.items():
        # Calculate Derivatives
        dissipation_results = \
            single_dissipation_from_dict_or_world_instance(
                host_object, target_object,
                rock_viscosity, rock_shear, rheology=rheo_name_,
                complex_compliance_inputs=rheo_input_,
                eccentricity=eccentricity, obliquity=obliquity,
                orbital_frequency=orbital_freq, 
                spin_frequency=spin_frequency,
                max_tidal_order_l=tidal_order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                fixed_dt=target_object['fixed_dt'],
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

        tidal_heating_targ = dissipation_results['tidal_heating']
        dspin_dt_targ = dissipation_results['spin_rate_derivative']
        de_dt = dissipation_results['eccentricity_derivative']
        da_dt = dissipation_results['semi_major_axis_derivative']

        # Prep for log plotting
        dspin_dt_targ_pos, dspin_dt_targ_neg = neg_array_for_log_plot(dspin_dt_targ)
        de_dt_pos, de_dt_neg = neg_array_for_log_plot(de_dt)

        # Plot results
        eccen_ax.plot(spin_period, de_dt_neg, c=blue, ls=line_style)
        eccen_ax.plot(spin_period, de_dt_pos, c=red, ls=line_style)
        heating_ax.plot(spin_period, tidal_heating_targ, c=red, ls=line_style, label=rheo_nice_name)
    
    # Add Legend
    heating_ax.legend(loc='best')
    
    plt.show()
    

In [10]:
cpl_ctl_compare(TRAPPIST1_e, TRAPPIST1, obliquity=0., eccentricity=0.4,
                target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                use_nsr=True, eccentricity_truncation=10,
                add_save_name=None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
cpl_ctl_compare(TRAPPIST1_e, TRAPPIST1, obliquity=0., eccentricity=0.1,
                target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                use_nsr=False,
                add_save_name=None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## NSR Plots

### Truncation Level Differences

In [12]:
def truncation_level_plots(target_object, host_object, obliquity=0.,
                           host_k2=0.33, host_q=5000.,
                           target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                           zpoints=np.linspace(-4, 4, 60), zticks=(-4, -2, 0, 2, 4),
                           ztick_names = ('$-10^{4}$', '$-10^{2}$', '$0$', '$10^{2}$', '$10^{4}$'),
                           eccentricity_truncations=((2, 4), (2, 14), (2, 18), (2, 22)),
                           truncation_names=(None, None, None, None),
                           add_save_name: str = None, slice_spin_to_orb: float = 1.,
                           save_fig=auto_save_plots_ck.value):
    
    print('This plot can take a while the first time it is run after a restart (functions are compiling).')
    time_i = time.time()
    # Pull out data
    constant_orbital_freq = target_object['orbital_freq']
    constant_host_spin_rate = constant_orbital_freq
    host_obliquity = 0.
    constant_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])

    # Domains
    oned_n, twod_n = get_resolution()
    eccentricity_domain = np.logspace(-2, 0., twod_n)
    spin_scale_domain = np.linspace(0., 3., twod_n)
    #    Make sure to hit resonances
    spin_scale_domain_res = np.asarray([0.5, 1., 1.5, 2., 2.5, 3.])
    spin_scale_domain = np.concatenate((spin_scale_domain, spin_scale_domain_res))
    spin_scale_domain = np.sort(spin_scale_domain)
    spin_domain = spin_scale_domain * constant_orbital_freq
    eccentricity, spin_rate = np.meshgrid(eccentricity_domain, spin_domain)
    shape = eccentricity.shape
    eccentricity = eccentricity.flatten()
    spin_rate = spin_rate.flatten()

    # Make sure that all input arrays have the correct shape
    x = eccentricity_domain
    y = spin_scale_domain
    obliquity *= np.ones_like(eccentricity)
    host_obliquity *= np.ones_like(eccentricity)
    constant_orbital_freq *= np.ones_like(spin_rate)
    constant_semi_major_axis *= np.ones_like(spin_rate)
    constant_host_spin_rate *= np.ones_like(constant_orbital_freq)
    #   Find spin_sync index
    sync_index = find_nearest(spin_scale_domain, slice_spin_to_orb)

    # Cases that are plotted (must be equal to 4, must have the format (order-l, eccentricity_trunc)
    case_line_styles = ['-', '--', '-.', ':']
    case_names = list()
    for case, provided_name in zip(eccentricity_truncations, truncation_names):
        if provided_name is not None:
            case_names.append(provided_name)
        else:
            # Use the truncation information to make a case name.
            order_l, trunc_level = case
            if order_l == 2:
                case_names.append('$\\mathcal{O}(e^{' + f'{trunc_level}' + '}$)')
            else:
                case_names.append('$l = ' + order_l + '\\mathcal{O}(e^{' + f'{trunc_level}' + '}$)')

    # Setup plots
    #    Contour Figure
    fig_contours = plt.figure(figsize=(6.75*1.75, 4.8), constrained_layout=True)
    ratios = (.249, .249, .249, .249, .02)
    gs_contours = GridSpec(1, 5, figure=fig_contours, width_ratios=ratios)
    case_contour_axes = [fig_contours.add_subplot(gs_contours[0, i]) for i in range(4)]
    colorbar_ax = fig_contours.add_subplot(gs_contours[0, 4])

    #    Spin-sync Figure
    fig_sync, sync_axes = plt.subplots(ncols=2, figsize=(1.5 * 6.4, 4.8))
    fig_sync.subplots_adjust(wspace=0.3)
    fig_sync.suptitle('$\\dot{\\theta} / n =' + f'{slice_spin_to_orb:0.1f}$', fontsize=16)
    sync_heating_ax = sync_axes[0]
    sync_eccen_ax = sync_axes[1]

    # Labels
    for ax in [sync_heating_ax, sync_eccen_ax] + case_contour_axes:
        if ax in [sync_heating_ax, sync_eccen_ax]:
            ax.set(xlabel='Eccentricity', xscale='linear')
        else:
            ax.set(xlabel='Eccentricity', xscale='log')
        
    for ax in fig_contours.get_axes():
        ax.label_outer()

    case_contour_axes[0].set(ylabel='$\\dot{\\theta} \\; / \\; n$')
    sync_heating_ax.set(ylabel='Tidal Heating [W]', yscale='log')
    sync_eccen_ax.set(ylabel='$\\dot{e}$ [yr$^{-1}$]', yscale='log')

    for case_i, (order_l, eccentricity_trunc) in enumerate(eccentricity_truncations):
        
        # User may provide 'CPL' or 'CTL' in the "order_l" variable. Check for that here.
        if type(order_l) == str:
            if order_l.lower() == 'cpl':
                rheo_color, rheo_model_name, rheo_input = rheologies['CPL']
            elif order_l.lower() == 'ctl':
                rheo_color, rheo_model_name, rheo_input = rheologies['CTL']
            else:
                raise KeyError('Unknown Model')
                
            # CPL and CTL use tidal order 2 only
            order_l = 2
        else:
            rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]
        
        # Calculate Derivatives
        dissipation_results = \
            single_dissipation_from_dict_or_world_instance(
                host_object, target_object,
                target_viscosity, target_shear, rheology=rheo_model_name,
                complex_compliance_inputs=rheo_input,
                eccentricity=eccentricity, obliquity=obliquity,
                orbital_frequency=constant_orbital_freq, 
                spin_frequency=spin_rate,
                max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_trunc,
                use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                fixed_dt=target_object['fixed_dt'],
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))
        
        tidal_heating_targ = dissipation_results['tidal_heating']
        dspin_dt_targ = dissipation_results['spin_rate_derivative']
        de_dt = dissipation_results['eccentricity_derivative']
        da_dt = dissipation_results['semi_major_axis_derivative']

        # Prep for log plotting
        dspin_dt_targ_pos, dspin_dt_targ_neg = neg_array_for_log_plot(dspin_dt_targ)
        de_dt_pos, de_dt_neg = neg_array_for_log_plot(de_dt)

        # Make data Symmetric Log (for negative logscale plotting)
        logpos = np.log10(np.copy(dspin_dt_targ_pos))
        logpos[logpos < 0.] = 0.
        negative_index = ~np.isnan(dspin_dt_targ_neg)
        logneg = np.log10(dspin_dt_targ_neg[negative_index])
        logneg[logneg < 0.] = 0.
        dspin_dt_targ_combo = logpos
        dspin_dt_targ_combo[negative_index] = -logneg
        
        # Reshape
        dspin_dt_targ_combo = dspin_dt_targ_combo.reshape(shape)
        tidal_heating_targ = tidal_heating_targ.reshape(shape)
        dspin_dt_targ = dspin_dt_targ.reshape(shape)
        de_dt = de_dt.reshape(shape)
        de_dt_neg = de_dt_neg.reshape(shape)
        de_dt_pos = de_dt_pos.reshape(shape)

        # Plot Contours
        case_name = case_names[case_i]
        contour_ax = case_contour_axes[case_i]
        contour_ax.set(title=case_name)
        cb_data = contour_ax.contourf(x, y, dspin_dt_targ_combo, zpoints, cmap=vik_map)
    
        for c in cb_data.collections:
            c.set_edgecolor("face")
        
        # Plot Spin Sync
        case_style = case_line_styles[case_i]
        #    Find sync data
        tidal_heating_sync = tidal_heating_targ[sync_index, :]
        de_dt_neg_sync = de_dt_neg[sync_index, :]
        de_dt_pos_sync = de_dt_pos[sync_index, :]
        #    Plot
        sync_heating_ax.plot(x, tidal_heating_sync, c=red, ls=case_style, label=case_name)
        sync_eccen_ax.plot(x, de_dt_neg_sync, c=blue, ls=case_style, label=case_name)
        sync_eccen_ax.plot(x, de_dt_pos_sync, c=red, ls=case_style)
        
        print(f'Case {case_i} completed.')

    # Add Colorbar
    cb = plt.colorbar(cb_data, pad=0.03, cax=colorbar_ax, ticks=zticks)
    spaces = '$' + '\\; '*14 + '$'
    cb.set_label('Decelerating ' + spaces + 'Accelerating\n$\\ddot{\\theta}$ [deg yr$^{-2}$]')
    cb.ax.set_yticklabels(ztick_names)
    
    # Add Spin-Sync Legend
    custom_lines = [Line2D([0], [0], color='k', lw=2, ls=style) for style in case_line_styles]
    sync_heating_ax.legend(custom_lines, case_names, loc='upper left')
    
    # Add Grid lines
    sync_heating_ax.grid(axis='x', which='major', alpha=0.5, ls='--')
    sync_heating_ax.grid(axis='x', which='minor', alpha=0.45, ls='-.')
    sync_heating_ax.set(ylim=(1e14, 1e21))
    sync_eccen_ax.grid(axis='x', which='major', alpha=0.5, ls='--')
    sync_eccen_ax.grid(axis='x', which='minor', alpha=0.45, ls='-.')
    sync_eccen_ax.set(ylim=(1e-11, 2e-7))

    # Add Annotations
    #    3:2 Arrows
    arrow_ax = case_contour_axes[0]
    arrow_ax.annotate(
            '', xy=(.9, .75), xytext=(.9, .51), xycoords='axes fraction', textcoords='axes fraction',
            arrowprops={'arrowstyle': '<-', 'lw': 2})
    arrow_ax.annotate(
            '', xy=(.9, .15), xytext=(.9, .49), xycoords='axes fraction', textcoords='axes fraction',
            arrowprops={'arrowstyle': '<-', 'lw': 2})
    arrow_ax.text(.27, .23, 'Accelerating\nSpin-Rate', fontsize=12, ha='center')
    arrow_ax.text(.27, 2.35, 'Decelerating\nSpin-Rate', fontsize=12, ha='center')
    arrow_ax.text(.07, 1.65, 'Circularization &\nResonance Fall-Off', fontsize=12, ha='center')

    #    3:2 R to L
    arrow_ax.annotate(
            '', xy=(.85, .53), xytext=(.5, .53), xycoords='axes fraction', textcoords='axes fraction',
            arrowprops={'arrowstyle': '<-', 'lw': 2})
    arrow_ax.annotate(
            '', xy=(.4, .53), xytext=(.25, .34), xycoords='axes fraction', textcoords='axes fraction',
            arrowprops={'arrowstyle': '<-', 'lw': 2})

    #    Add Mercury Marker
    merc_ax = case_contour_axes[2]
    merc_x = 0.205630
    merc_y = 1.5
    merc_ax.text(.055, merc_y * 1.05, 'Mercury', fontsize=12)
    merc_ax.plot([merc_x], [merc_y], 'kx')
    
    time_f = time.time() - time_i
    print(f'Calculations and plot reinit took {time_f:0.1f} seconds.')

    if save_fig:
        save_name = analysis_version_save + 'TruncationPlot_' + target_object['name'] + '_' + host_object['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        fig_contours.savefig(save_name + '_Contours.pdf')
        fig_sync.tight_layout(rect=[0, 0, 1, 0.95])
        fig_sync.savefig(save_name + '_SpinSync.pdf')
    
    plt.show()
    return fig_contours, fig_sync

#### Basic Truncation Comparison - No Obliquity

In [13]:
truncation_level_plots(TRAPPIST1_e, TRAPPIST1, obliquity=0.,
                       target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                       add_save_name=None)

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Case 0 completed.
Case 1 completed.
Case 2 completed.
Case 3 completed.
Calculations and plot reinit took 2.0 seconds.


(<Figure size 1181.25x480 with 5 Axes>, <Figure size 960x480 with 2 Axes>)

In [14]:
truncation_level_plots(TRAPPIST1_e, TRAPPIST1, obliquity=0.,
                       target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='CTL',
                       add_save_name=None)

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Case 0 completed.
Case 1 completed.
Case 2 completed.
Case 3 completed.
Calculations and plot reinit took 1.4 seconds.


(<Figure size 1181.25x480 with 5 Axes>, <Figure size 960x480 with 2 Axes>)

In [15]:
truncation_level_plots(TRAPPIST1_e, TRAPPIST1, obliquity=0.,
                       target_viscosity=rock_viscosity/100, target_shear=rock_shear, target_rheo='CPL',
                       add_save_name=None)

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Case 0 completed.
Case 1 completed.
Case 2 completed.
Case 3 completed.
Calculations and plot reinit took 1.3 seconds.


(<Figure size 1181.25x480 with 5 Axes>, <Figure size 960x480 with 2 Axes>)

#### Basic Truncation Comparison - 20 deg Obliquity

In [16]:
truncation_level_plots(TRAPPIST1_e, TRAPPIST1, obliquity=np.radians(35.),
                       target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                       add_save_name='Oblquity35')

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Case 0 completed.
Case 1 completed.
Case 2 completed.
Case 3 completed.
Calculations and plot reinit took 1.9 seconds.


(<Figure size 1181.25x480 with 5 Axes>, <Figure size 960x480 with 2 Axes>)

#### Comparison to CPL/CTL

In [17]:
truncation_level_plots(TRAPPIST1_e, TRAPPIST1, obliquity=0.,
                       host_k2=0.33, host_q=5000.,
                       target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                       zpoints=np.linspace(-4, 4, 60), zticks=(-4, -2, 0, 2, 4),
                       ztick_names = ('$-10^{4}$', '$-10^{2}$', '$0$', '$10^{2}$', '$10^{4}$'),
                       eccentricity_truncations=((2, 6), (2, 20), ('CPL', 6), ('CTL', 6)),
                       truncation_names=(None, None, 'CPL', 'CTL'),
                       add_save_name=None, slice_spin_to_orb=1.5,
                       save_fig=auto_save_plots_ck.value)

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Case 0 completed.
Case 1 completed.
Case 2 completed.
Case 3 completed.
Calculations and plot reinit took 1.2 seconds.


(<Figure size 1181.25x480 with 5 Axes>, <Figure size 960x480 with 2 Axes>)

### Single Truncation Contours

In [18]:
def single_contour_plot(target_object, host_object, obliquity=0.,
                        target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                        host_k2=0.33, host_q=5000.,
                        order_l: int = 2, eccentricity_truncation: int = 10,
                        zpoints=np.linspace(-5, 5, 60), zticks=(-5, -3, -1, 0, 1, 3, 5),
                        ztick_names = ('$-10^{5}$', '$-10^{3}$', '$-10^{1}$', '$0$', '$10^{1}$', '$10^{3}$', '$10^{5}$'),   
                        draw_resonance_lines: bool = True,
                        title = None, add_save_name: str = None, save_fig=auto_save_plots_ck.value):
    
    print('This plot can take a while the first time it is run after a restart (functions are compiling).')
    # Pull out data
    constant_orbital_freq = target_object['orbital_freq']
    constant_host_spin_rate = constant_orbital_freq
    host_obliquity = 0.
    constant_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])

    # Domains
    oned_n, twod_n = get_resolution()
    eccentricity_domain = np.logspace(-2, 0., twod_n)
    spin_scale_domain = np.linspace(0., 3., twod_n+1)
    #    Make sure to hit resonances
    spin_scale_domain_res = np.asarray([0.5, 1., 1.5, 2., 2.5, 3.])
    spin_scale_domain = np.concatenate((spin_scale_domain, spin_scale_domain_res))
    spin_scale_domain = np.sort(spin_scale_domain)
    spin_domain = spin_scale_domain * constant_orbital_freq
    eccentricity, spin_rate = np.meshgrid(eccentricity_domain, spin_domain)
    shape = eccentricity.shape
    eccentricity = eccentricity.flatten()
    spin_rate = spin_rate.flatten()

    # Make sure that all input arrays have the correct shape
    x = eccentricity_domain
    y = spin_scale_domain
    min_y = np.min(y)
    obliquity *= np.ones_like(eccentricity)
    host_obliquity *= np.ones_like(eccentricity)
    constant_orbital_freq *= np.ones_like(spin_rate)
    constant_semi_major_axis *= np.ones_like(spin_rate)
    constant_host_spin_rate *= np.ones_like(constant_orbital_freq)
    #   Find spin_sync index
    sync_index = find_nearest(spin_scale_domain, 1.)
    
    # Single plot to compare resonance heating rates
    fig_single, ax_sig = plt.subplots(figsize=(6.4, 4.8), constrained_layout=True)
    ax_sig.set(xlabel='Eccentricity', xscale='log', ylabel='Tidal Heating [W]', yscale='log')

    # Setup plots
    #    Contour Figure
    fig_contour, ax = plt.subplots(figsize=(6.4, 4.8), constrained_layout=True)

    # Labels
    ax.set(xlabel='Eccentricity', xscale='log', ylabel='$\\dot{\\theta} \\; / \\; n$')

    # For this analysis we always assume the host body is dissipating like a constant fixed q.    
    # Need to build a new tidal calculation function based on this truncation level
    rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]
        
    # Calculate Derivatives
    dissipation_results = \
        single_dissipation_from_dict_or_world_instance(
            host_object, target_object,
            target_viscosity, target_shear, rheology=rheo_model_name,
            complex_compliance_inputs=rheo_input,
            eccentricity=eccentricity, obliquity=obliquity,
            orbital_frequency=constant_orbital_freq, 
            spin_frequency=spin_rate,
            max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
            use_obliquity=True, tidal_scale=target_object['tidal_scale'],
            fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
            fixed_dt=target_object['fixed_dt'],
            da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
            dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

    tidal_heating_targ = dissipation_results['tidal_heating']
    dspin_dt_targ = dissipation_results['spin_rate_derivative']
    de_dt = dissipation_results['eccentricity_derivative']
    da_dt = dissipation_results['semi_major_axis_derivative']

    # Prep for log plotting
    dspin_dt_targ_pos, dspin_dt_targ_neg = neg_array_for_log_plot(dspin_dt_targ)
    
    tidal_heating_targ_log = np.log10(tidal_heating_targ)
    de_dt_pos, de_dt_neg = neg_array_for_log_plot(de_dt)

    # Make data Symmetric Log (for negative logscale plotting)
    logpos = np.log10(np.copy(dspin_dt_targ_pos))
    logpos[logpos < 0.] = 0.
    negative_index = ~np.isnan(dspin_dt_targ_neg)
    logneg = np.log10(dspin_dt_targ_neg[negative_index])
    logneg[logneg < 0.] = 0.
    dspin_dt_targ_combo = logpos
    dspin_dt_targ_combo[negative_index] = -logneg
    
    # Reshape
    dspin_dt_targ_combo = dspin_dt_targ_combo.reshape(shape)
    de_dt_pos = de_dt_pos.reshape(shape)
    de_dt_neg = de_dt_neg.reshape(shape)
    tidal_heating_targ = tidal_heating_targ.reshape(shape)
    dspin_dt_targ = dspin_dt_targ.reshape(shape)
    de_dt = de_dt.reshape(shape)

    # Plot Contours
    cb_data = ax.contourf(x, y, dspin_dt_targ_combo, zpoints, cmap=vik_map)
    
    for c in cb_data.collections:
        c.set_edgecolor("face")
    
    # Add Resonances to Signle Plot
    resonances = {'-': 1., '--': 1.5, '-.': 2., ':': 2.5}
    for res_ls, res in resonances.items():
        y_index = find_nearest(y, res)
        heating = tidal_heating_targ[y_index, :]
        res_label = '$\\dot{\\theta}/n=' + f'{res}' + '$'
        ax_sig.plot(x, heating, ls=res_ls, label=res_label)
    ax_sig.legend(loc='best')

    # Print minimum eccentricity for a higher order lock
    print('Minimum Eccentricities for Resonance Locks:')
    for resonance in spin_scale_domain_res:
        y_coord = find_nearest(y, resonance-0.006)
        y_value = y[y_coord]
        dspin_dt_at_res = dspin_dt_targ_combo[y_coord, :]
        cross_coord = find_nearest(dspin_dt_at_res, 0.)
        deriv_value = dspin_dt_targ_combo[y_coord, cross_coord]
        print(f'\tO/n = {resonance}')
        print(f'\tO/n real = {y_value:0.3f}')
        if abs(deriv_value) > 0.5:
            print('\tNo Lock')
        else:
            min_e = eccentricity_domain[cross_coord]
            if draw_resonance_lines:
                ax.plot([min_e, min_e], [min_y, y_value + 0.01], ls=':', c='k')
                ax_sig.axvline(x=min_e, ls=':', c='k')
            print(f'\tdO/dt = {deriv_value:0.3f}')
            print(f'\te_min = {min_e:0.5f}')
        print('-------')

    # Add Colorbar
    cb = plt.colorbar(cb_data, pad=0.03, ticks=zticks)
    spaces = '$' + '\\; '*19 + '$'
    cb.set_label('Decelerating ' + spaces + 'Accelerating\n$\\ddot{\\theta}$ [deg yr$^{-2}$]')
    cb.ax.set_yticklabels(ztick_names)
    
    # Add title
    if title is not None:
        ax.set(title=title)
    
    if save_fig:
        save_name = analysis_version_save + '_SingleContourPlot_' + target_object['name'] + '_' + host_object['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        fig_contour.savefig(save_name + 'Contour.pdf')
        fig_single.savefig(save_name + 'SingleHeating.pdf')

#### Regular

In [57]:
single_contour_plot(TRAPPIST1_e, TRAPPIST1, obliquity=0.,
                    target_viscosity=rock_viscosity/100, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                    zpoints=np.linspace(-5, 5, 40), zticks=(-5, -3, -1, 0, 1, 3, 5),
                    title=None, add_save_name='Regular')

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Minimum Eccentricities for Resonance Locks:
	O/n = 0.5
	O/n real = 0.498
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.0
	O/n real = 0.996
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.5
	O/n real = 1.493
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.0
	O/n real = 1.991
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.5
	O/n real = 2.489
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 3.0
	O/n real = 3.000
	dO/dt = nan
	e_min = 1.00000
-------


#### Low Viscosity

In [56]:
single_contour_plot(TRAPPIST1_e, TRAPPIST1, obliquity=0.,
                    target_viscosity=ice_viscosity, target_shear=ice_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                    zpoints=np.linspace(-5, 5, 60), zticks=(-5, -3, -1, 0, 1, 3, 5),
                    draw_resonance_lines=False,
                    title='Low Viscosity', add_save_name='Viscosity1e14')

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Minimum Eccentricities for Resonance Locks:
	O/n = 0.5
	O/n real = 0.498
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.0
	O/n real = 0.996
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.5
	O/n real = 1.493
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.0
	O/n real = 1.991
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.5
	O/n real = 2.489
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 3.0
	O/n real = 3.000
	dO/dt = nan
	e_min = 1.00000
-------


#### Proxima

In [55]:
single_contour_plot(Proxima_b, Proxima, obliquity=0.,
                    target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                    zpoints=np.linspace(-5, 5, 40), zticks=(-5, -3, -1, 0, 1, 3, 5),
                    title='Proxima b', add_save_name='Regular')

This plot can take a while the first time it is run after a restart (functions are compiling).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Minimum Eccentricities for Resonance Locks:
	O/n = 0.5
	O/n real = 0.498
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.0
	O/n real = 0.996
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.5
	O/n real = 1.493
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.0
	O/n real = 1.991
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.5
	O/n real = 2.489
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 3.0
	O/n real = 3.000
	dO/dt = nan
	e_min = 1.00000
-------


### Double Contour Plots

In [22]:
def double_contour_plot(target_object, host_object, obliquity=0.,
                        target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                        host_k2=0.33, host_q=5000., host_obliquity=0.,
                        order_l: int = 2, eccentricity_truncation: int = 10,
                        zpoints=np.linspace(-5, 5, 60), zticks=(-5, -3, -1, 0, 1, 3, 5),
                        ztick_names = ('$-10^{5}$', '$-10^{3}$', '$-10^{1}$', '$0$', '$10^{1}$', '$10^{3}$', '$10^{5}$'), 
                        draw_resonance_lines: bool = True,
                        text_2x: float = None, text_2y: float = None, text_2: str = None,
                        orbital_period_replacement: float = None,
                        title_1 = None, title_2 = None, add_save_name: str = None, save_fig=auto_save_plots_ck.value):

    # Pull out data
    constant_orbital_freq = target_object['orbital_freq']
    if orbital_period_replacement is not None:
        old_orbital_period = rads2days(constant_orbital_freq)
        old_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])
        print(f'Old Orbital Period: {old_orbital_period:0.2f} Days')
        print(f'Old Semi-Major Axis: {old_semi_major_axis/1000:0.1f} km')
        constant_orbital_freq = days2rads(orbital_period_replacement)
        constant_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])
    else:
        constant_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])
    constant_host_spin_rate = constant_orbital_freq

    print(f'Orbital Period: {rads2days(constant_orbital_freq):0.2f} Days')
    print(f'Semi-Major Axis: {constant_semi_major_axis/1000:0.1f} km')

    # Domains
    oned_n, twod_n = get_resolution()
    eccentricity_domain = np.logspace(-2, 0., twod_n)
    spin_scale_domain = np.linspace(0., 3., twod_n)
    #    Make sure to hit resonances
    spin_scale_domain_res = np.asarray([0.5, 1., 1.5, 2., 2.5, 3.])
    spin_scale_domain = np.concatenate((spin_scale_domain, spin_scale_domain_res))
    spin_scale_domain = np.sort(spin_scale_domain)
    spin_domain = spin_scale_domain * constant_orbital_freq
    eccentricity, spin_rate = np.meshgrid(eccentricity_domain, spin_domain)
    shape = eccentricity.shape
    eccentricity = eccentricity.flatten()
    spin_rate = spin_rate.flatten()

    # Make sure that all input arrays have the correct shape
    x = eccentricity_domain
    y = spin_scale_domain
    min_y = np.min(y)
    obliquity *= np.ones_like(eccentricity)
    host_obliquity *= np.ones_like(eccentricity)
    constant_orbital_freq *= np.ones_like(spin_rate)
    constant_semi_major_axis *= np.ones_like(spin_rate)
    constant_host_spin_rate *= np.ones_like(constant_orbital_freq)
    #   Find spin_sync index
    sync_index = find_nearest(spin_scale_domain, 1.)
    
    # Find rheology information
    rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]

    # Setup plots
    #    Contour Figure
    fig_contour = plt.figure(figsize=(6.4*1.75, 4.8), constrained_layout=True)
    ratios = (0.49, 0.49, 0.02)
    gs = GridSpec(1, 3, figure=fig_contour, width_ratios=ratios, wspace=0.2)
    axes = [fig_contour.add_subplot(gs[0, i]) for i in range(2)]
    colorbar_ax = fig_contour.add_subplot(gs[0, 2])
    
    for ax in fig_contour.get_axes():
        ax.label_outer()
    
    for ax_i, ax in enumerate(axes):
        
         # Labels
        ax.set(xlabel='Eccentricity', xscale='log', ylim=(0., spin_scale_domain_res[-1]))
        
        if ax_i == 0:    
            # Regular Plot
            ax.set(ylabel='$\\dot{\\theta} \\; / \\; n$')
            # Add title
            if title_1 is not None:
                ax.set(title=title_1)
        
            # Calculate Derivatives
            dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=eccentricity, obliquity=0.,
                    orbital_frequency=constant_orbital_freq, 
                    spin_frequency=spin_rate,
                    max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

            tidal_heating_targ = dissipation_results['tidal_heating']
            dspin_dt_targ = dissipation_results['spin_rate_derivative']
            de_dt = dissipation_results['eccentricity_derivative']
            da_dt = dissipation_results['semi_major_axis_derivative']
            
        else:
            # Add title
            if title_2 is not None:
                ax.set(title=title_2)
            # Add Text
            if text_2 is not None:
                ax.text(text_2x, text_2y, text_2, fontsize=14, ha='center')
            
            # Calculate Derivatives
            dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=eccentricity, obliquity=obliquity,
                    orbital_frequency=constant_orbital_freq, 
                    spin_frequency=spin_rate,
                    max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

            tidal_heating_targ = dissipation_results['tidal_heating']
            dspin_dt_targ = dissipation_results['spin_rate_derivative']
            de_dt = dissipation_results['eccentricity_derivative']
            da_dt = dissipation_results['semi_major_axis_derivative']
            
        # Prep for log plotting
        dspin_dt_targ_pos, dspin_dt_targ_neg = neg_array_for_log_plot(dspin_dt_targ)
        de_dt_pos, de_dt_neg = neg_array_for_log_plot(de_dt)

        # Make data Symmetric Log (for negative logscale plotting)
        logpos = np.log10(np.copy(dspin_dt_targ_pos))
        logpos[logpos < 0.] = 0.
        negative_index = ~np.isnan(dspin_dt_targ_neg)
        logneg = np.log10(dspin_dt_targ_neg[negative_index])
        logneg[logneg < 0.] = 0.
        dspin_dt_targ_combo = logpos
        dspin_dt_targ_combo[negative_index] = -logneg
        
        # Reshape
        dspin_dt_targ_combo = dspin_dt_targ_combo.reshape(shape)
        de_dt_pos = de_dt_pos.reshape(shape)
        de_dt_neg = de_dt_neg.reshape(shape)
        tidal_heating_targ = tidal_heating_targ.reshape(shape)
        dspin_dt_targ = dspin_dt_targ.reshape(shape)
        de_dt = de_dt.reshape(shape)

        # Plot Contours
        cb_data = ax.contourf(x, y, dspin_dt_targ_combo, zpoints, cmap=vik_map)
        for c in cb_data.collections:
            c.set_edgecolor("face")

        # Print minimum eccentricity for a higher order lock
        print(f'Minimum Eccentricities for Resonance Locks for subplot {ax_i}:')
        for resonance in spin_scale_domain_res:
            y_coord = find_nearest(y, resonance-0.003)
            y_value = y[y_coord]
            dspin_dt_at_res = dspin_dt_targ_combo[y_coord, :]
            cross_coord = find_nearest(dspin_dt_at_res, 0.)
            deriv_value = dspin_dt_targ_combo[y_coord, cross_coord]
            print(f'\tO/n = {resonance}')
            print(f'\tO/n real = {y_value:0.3f}')
            if abs(deriv_value) > 0.5:
                print('\tNo Lock')
            else:
                min_e = eccentricity_domain[cross_coord]
                if draw_resonance_lines:
                    ax.plot([min_e, min_e], [min_y, y_value + 0.01], ls=':', c='k')
                print(f'\tdO/dt = {deriv_value:0.3f}')
                print(f'\te_min = {min_e:0.5f}')
            print('-------')

    # Add Colorbar    
    cb = plt.colorbar(cb_data, cax=colorbar_ax, pad=0.03, ticks=zticks)
    spaces = '$' + '\\; '*19 + '$'
    cb.set_label('Decelerating ' + spaces + 'Accelerating\n$\\ddot{\\theta}$ [deg yr$^{-2}$]')
    cb.ax.set_yticklabels(ztick_names)
    
    if save_fig:
        save_name = analysis_version_save + '_DoubleContourPlot_' + target_object['name'] + '_' + host_object['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        fig_contour.savefig(save_name + '.pdf')

#### Obliquity Impact on NSR

In [54]:
double_contour_plot(TRAPPIST1_e, TRAPPIST1, obliquity=np.radians(35.),
                    target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                        zpoints=np.linspace(-4, 4, 60), zticks=(-4, -2, 0, 2, 4),
                        ztick_names = ('$-10^{4}$', '$-10^{2}$', '$0$', '$10^{2}$', '$10^{4}$'),
                    title_1='0$^{\\degree}$ Obliquity',
                    title_2='35$^{\\degree}$ Obliquity',
                    text_2x=5e-2, text_2y=2.1, text_2='2:1 Resonance at Low-$e$',
                    add_save_name='Obliquity35')

Orbital Period: 6.45 Days
Semi-Major Axis: 4380700.0 km


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Minimum Eccentricities for Resonance Locks for subplot 0:
	O/n = 0.5
	O/n real = 0.500
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.0
	O/n real = 1.000
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 1.5
	O/n real = 1.500
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.0
	O/n real = 2.000
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 2.5
	O/n real = 2.500
	dO/dt = nan
	e_min = 1.00000
-------
	O/n = 3.0
	O/n real = 3.000
	dO/dt = nan
	e_min = 1.00000
-------
Minimum Eccentricities for Resonance Locks for subplot 1:
	O/n = 0.5
	O/n real = 0.500
	dO/dt = 0.319
	e_min = 0.01000
-------
	O/n = 1.0
	O/n real = 1.000
	dO/dt = 0.000
	e_min = 0.01000
-------
	O/n = 1.5
	O/n real = 1.500
	dO/dt = -0.000
	e_min = 0.24911
-------
	O/n = 2.0
	O/n real = 2.000
	dO/dt = -0.000
	e_min = 0.39318
-------
	O/n = 2.5
	O/n real = 2.500
	dO/dt = -0.000
	e_min = 0.47389
-------
	O/n = 3.0
	O/n real = 3.000
	dO/dt = -0.000
	e_min = 0.53670
-------


### Obliquity Contour Plots

In [24]:
def double_contour_obliquity_plot(target_object, host_object, eccentricity=0.,
                                  target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                                  host_k2=0.33, host_q=5000.,
                                  order_l: int = 2, eccentricity_truncation: int = 20,
                                  zpoints=np.linspace(-5, 5, 60),
                                  zticks=(-5, -3, -1, 0, 1, 3, 5),
                                  ztick_names = ('$-10^{5}$', '$-10^{3}$', '$-10^{1}$', '$0$', '$10^{1}$', '$10^{3}$', '$10^{5}$'),
                                  text_2x: float = None, text_2y: float = None, text_2: str = None,
                                  orbital_period_replacement: float = None,
                                  title_1 = None, title_2 = None, add_save_name: str = None, save_fig=auto_save_plots_ck.value):
    
    print('This plot can take a while the first time it is run after a restart (functions are compiling).')
    # Pull out data
    constant_orbital_freq = target_object['orbital_freq']
    if orbital_period_replacement is not None:
        old_orbital_period = rads2days(constant_orbital_freq)
        old_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])
        print(f'Old Orbital Period: {old_orbital_period:0.2f} Days')
        print(f'Old Semi-Major Axis: {old_semi_major_axis/1000:0.1f} km')
        constant_orbital_freq = days2rads(orbital_period_replacement)
        constant_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])
    else:
        constant_semi_major_axis = orbital_motion2semi_a(constant_orbital_freq, host_object['mass'], target_object['mass'])
    
    print(f'Orbital Period: {rads2days(constant_orbital_freq):0.2f} Days')
    print(f'Semi-Major Axis: {constant_semi_major_axis/1000:0.1f} km')

    # Domains
    oned_n, twod_n = get_resolution()
    obliquity_domain_deg = np.linspace(0., 180., twod_n)
    obliquity_domain = np.radians(obliquity_domain_deg)
    spin_scale_domain = np.linspace(0., 3., twod_n)
    #    Make sure to hit resonances
    spin_scale_domain_res = np.asarray([0.5, 1., 1.5, 2., 2.5, 3.])
    spin_scale_domain = np.concatenate((spin_scale_domain, spin_scale_domain_res))
    spin_scale_domain = np.sort(spin_scale_domain)
    spin_domain = spin_scale_domain * constant_orbital_freq
    obliquity, spin_rate = np.meshgrid(obliquity_domain, spin_domain)
    shape = obliquity.shape
    obliquity = obliquity.flatten()
    spin_rate = spin_rate.flatten()

    # Make sure that all input arrays have the correct shape
    x = obliquity_domain_deg
    y = spin_scale_domain
    min_y = np.min(y)
    constant_orbital_freq *= np.ones_like(spin_rate)
    constant_semi_major_axis *= np.ones_like(spin_rate)
    #   Find spin_sync index
    sync_index = find_nearest(spin_scale_domain, 1.)
    
    # Find rheology information
    rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]

    # Setup plots
    #    Contour Figure
    fig_contour = plt.figure(figsize=(6.4*1.75, 4.8), constrained_layout=True)
    ratios = (0.49, 0.49, 0.02)
    gs = GridSpec(1, 3, figure=fig_contour, width_ratios=ratios, wspace=0.2)
    axes = [fig_contour.add_subplot(gs[0, i]) for i in range(2)]
    colorbar_ax = fig_contour.add_subplot(gs[0, 2])
    
    for ax in fig_contour.get_axes():
        ax.label_outer()
    
    for ax_i, ax in enumerate(axes):
        
         # Labels
        ax.set(xlabel='Obliquity [deg.]', xscale='linear', ylim=(spin_scale_domain_res[0], spin_scale_domain_res[-1]))
        
        if ax_i == 0:
            # Add title
            if title_1 is not None:
                ax.set(title=title_1) 
            # Add Text
            if text_2 is not None:
                ax.text(text_2x, text_2y, text_2, fontsize=14, ha='center')
            
            # Calculate Derivatives
            dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=np.zeros_like(obliquity), obliquity=obliquity,
                    orbital_frequency=constant_orbital_freq, 
                    spin_frequency=spin_rate,
                    max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

            tidal_heating_targ = dissipation_results['tidal_heating']
            dspin_dt_targ = dissipation_results['spin_rate_derivative']
            de_dt = dissipation_results['eccentricity_derivative']
            da_dt = dissipation_results['semi_major_axis_derivative']
            
        else:
            # Add title
            if title_2 is not None:
                ax.set(title=title_2)
            
            # Calculate Derivatives
            dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=eccentricity * np.ones_like(obliquity), obliquity=obliquity,
                    orbital_frequency=constant_orbital_freq, 
                    spin_frequency=spin_rate,
                    max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

            tidal_heating_targ = dissipation_results['tidal_heating']
            dspin_dt_targ = dissipation_results['spin_rate_derivative']
            de_dt = dissipation_results['eccentricity_derivative']
            da_dt = dissipation_results['semi_major_axis_derivative']

        # Prep for log plotting
        dspin_dt_targ_pos, dspin_dt_targ_neg = neg_array_for_log_plot(dspin_dt_targ)
        de_dt_pos, de_dt_neg = neg_array_for_log_plot(de_dt)
        
        # Make data Symmetric Log (for negative logscale plotting)
        logpos = np.log10(np.copy(dspin_dt_targ_pos))
        logpos[logpos < 0.] = 0.
        negative_index = ~np.isnan(dspin_dt_targ_neg)
        logneg = np.log10(dspin_dt_targ_neg[negative_index])
        logneg[logneg < 0.] = 0.
        dspin_dt_targ_combo = logpos
        dspin_dt_targ_combo[negative_index] = -logneg
        
        # Reshape
        dspin_dt_targ_combo = dspin_dt_targ_combo.reshape(shape)
        de_dt_pos = de_dt_pos.reshape(shape)
        de_dt_neg = de_dt_neg.reshape(shape)
        tidal_heating_targ = tidal_heating_targ.reshape(shape)
        dspin_dt_targ = dspin_dt_targ.reshape(shape)
        de_dt = de_dt.reshape(shape)

        # Plot Contours
        cb_data = ax.contourf(x, y, dspin_dt_targ_combo, zpoints, cmap=vik_map)
        for c in cb_data.collections:
            c.set_edgecolor("face")

    # Add Colorbar
    cb = plt.colorbar(cb_data, cax=colorbar_ax, pad=0.03, ticks=zticks)
    spaces = '$' + '\\; '*19 + '$'
    cb.set_label('Decelerating ' + spaces + 'Accelerating\n$\\ddot{\\theta}$ [deg yr$^{-2}$]')
    cb.ax.set_yticklabels(ztick_names)
        
    for ax in axes:
        ax.set_xticks([0, 45, 90, 135, 180])
    
    if save_fig:
        save_name = analysis_version_save + '_DoubleContourObliquityPlot_' + target_object['name'] + '_' + host_object['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        fig_contour.savefig(save_name + '.pdf')

#### Obliquity Contour for e=0 (left) and e=0.1 (right)

In [53]:
double_contour_obliquity_plot(TRAPPIST1_e, TRAPPIST1, eccentricity=0.1,
                    target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                    zpoints=np.linspace(-5, 5, 40), zticks=(-5, -3, -1, 0, 1, 3, 5),
                    title_1='$e\\;=\\;0$',
                    title_2='$e\\;=\\;0.1$',
                    text_2='2:1 Resonance Independent of $e$', text_2x=75, text_2y=2.1,
                    add_save_name='Eccentricity0.1')

This plot can take a while the first time it is run after a restart (functions are compiling).
Orbital Period: 6.45 Days
Semi-Major Axis: 4380700.0 km


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Obliquity Contour for e=0 (left) and e=0.3 (right)

In [52]:
double_contour_obliquity_plot(TRAPPIST1_e, TRAPPIST1, eccentricity=0.3,
                    target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                        zpoints=np.linspace(-4, 4, 100), zticks=(-4, -2, 0, 2, 4),
                        ztick_names = ('$-10^{4}$', '$-10^{2}$', '$0$', '$10^{2}$', '$10^{4}$'),
                    title_1='$e\\;=\\;0$',
                    title_2='$e\\;=\\;0.3$',
                    text_2='2:1 Resonance Independent of $e$', text_2x=75, text_2y=2.1,
                    add_save_name='Eccentricity0p3')

This plot can take a while the first time it is run after a restart (functions are compiling).
Orbital Period: 6.45 Days
Semi-Major Axis: 4380700.0 km


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Obliquity Contour for e=0 (left) and e=0.6 (right)

In [51]:
double_contour_obliquity_plot(TRAPPIST1_e, TRAPPIST1, eccentricity=0.6,
                    target_viscosity=rock_viscosity, target_shear=rock_shear, target_rheo='Sundberg-Cooper',
                    host_k2=0.33, host_q=5000.,
                    order_l=2, eccentricity_truncation=20,
                    zpoints=np.linspace(-5, 5, 40), zticks=(-5, -3, -1, 0, 1, 3, 5),
                    title_1='$e\\;=\\;0$',
                    title_2='$e\\;=\\;0.6$',
                    text_2='2:1 Resonance Independent of $e$', text_2x=75, text_2y=2.1,
                    add_save_name='Eccentricity0.6')

This plot can take a while the first time it is run after a restart (functions are compiling).
Orbital Period: 6.45 Days
Semi-Major Axis: 4380700.0 km


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Obliquity Domain - Eccentricity Levels

In [28]:
def obliquity_variation_eccentricity_truncation_1(target_object, host_object,
                        target_viscosity=5.0e14, target_shear=3.3e9, target_rheo='Sundberg-Cooper',
                        double_plot=True,
                        eccentricity=.3, add_save_name: str = None, save_fig=auto_save_plots_ck.value, different_e=False):
    
    # Find rheology information
    rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]

    # Setup Domain and Cases
    if different_e:
        cases = [(2, .0), (2, .1), (2, .3), (2, .6)]
        case_line_styles = ['-', '--', '-.', ':']
        case_names = ['$e=' + f'{trunc_level}' + '}$' if order_l == 2 
                      else '$l = ' + str(order_l) + 'e=' + f'{trunc_level}' + '}$'
                      for order_l, trunc_level in cases]
    else:
        cases = [(2, 2), (2, 6), (2, 10), (2, 20)]
        case_line_styles = ['-', '--', '-.', ':']
        case_names = ['$\\mathcal{O}(e^{' + f'{trunc_level}' + '}$)' if order_l == 2 
                      else '$l = ' + str(order_l) + '\\mathcal{O}(e^{' + f'{trunc_level}' + '}$)'
                      for order_l, trunc_level in cases]
    
    # Setup Domains
    oned_n, twod_n = get_resolution()
    obliquity_domain_deg = np.linspace(0, 360., oned_n)
    obliquity_domain = np.radians(obliquity_domain_deg)
    eccentricity *= np.ones_like(obliquity_domain)
    spin_rate = target_object['orbital_freq']
    orbital_freq = target_object['orbital_freq']
    spin_rate *= np.ones_like(obliquity_domain)
    orbital_freq *= np.ones_like(obliquity_domain)
    semi_major_axis = orbital_motion2semi_a(orbital_freq, host_object['mass'], target_object['mass'])
    
    # Setup plot
    if double_plot:
        fig, axes = plt.subplots(ncols=2, figsize=(6.4*1.75, 4.8), constrained_layout=True)
        ax0 = axes[0]
        ax1 = axes[1]
        ax0.set_title('$\\dot{\\theta} \\; = \\; n$')
        ax1.set_title('$\\dot{\\theta} \\; = \\; 2n$')
        
        fig2, axes2 = plt.subplots(ncols=2, figsize=(6.4*1.75, 4.8), constrained_layout=True)
        ax2_0 = axes2[0]
        ax2_1 = axes2[1]
        ax2_0.set_title('$\\dot{\\theta} \\; = \\; n$')
        ax2_1.set_title('$\\dot{\\theta} \\; = \\; 2n$')
        
    else:
        fig, ax0 = plt.subplots(ncols=1, figsize=(6.4, 4.8), constrained_layout=True)
        ax1 = None
        ax0.set_title('$\\dot{\\theta} \\; = \\; n$')
        axes = [ax0]
        
        fig2, ax2_0 = plt.subplots(ncols=1, figsize=(6.4, 4.8), constrained_layout=True)
        ax2_1 = None
        axes2 = [ax2_0]
        ax2_0.set_title('$\\dot{\\theta} \\; = \\; n$')
    
    for ax in fig.get_axes():
        ax.label_outer()
    
    for ax in fig2.get_axes():
        ax.label_outer()
    
    for ax in axes:
        if ax is ax0:
            ax.set(xlabel='Obliquity [deg.]', ylabel='Tidal Heating [W]')
        else:
            ax.set(xlabel='Obliquity [deg.]')
            
    for ax in axes2:
        if ax is ax2_0:
            ax.set(xlabel='Obliquity [deg.]', ylabel='Tidal Polar Torque [N m]')
        else:
            ax.set(xlabel='Obliquity [deg.]')
    
    max_heating = 0.
    min_heating = 1.e100
    max_torque = 0.
    min_torque = 1.e100
    for case_i, (max_order_l, eccentricity_trunc) in enumerate(cases):
        
        if different_e:
            eccentricity = eccentricity_trunc
            eccentricity_trunc = 20
        
        dissipation_results = \
            single_dissipation_from_dict_or_world_instance(
                host_object, target_object,
                target_viscosity, target_shear, rheology=rheo_model_name,
                complex_compliance_inputs=rheo_input,
                eccentricity=eccentricity, obliquity=obliquity_domain,
                orbital_frequency=orbital_freq, 
                spin_frequency=orbital_freq,
                max_tidal_order_l=max_order_l, eccentricity_truncation_lvl=eccentricity_trunc,
                use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                fixed_dt=target_object['fixed_dt'],
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

        tidal_heating_targ_pos = dissipation_results['tidal_heating']
        tidal_torque_pos = dissipation_results['tidal_torque']
        dspin_dt_targ_pos = dissipation_results['spin_rate_derivative']
        de_dt_pos = dissipation_results['eccentricity_derivative']
        da_dt_pos = dissipation_results['semi_major_axis_derivative']
        
        if double_plot:
            # Calculate tides: spin=-n
            dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=eccentricity, obliquity=obliquity_domain,
                    orbital_frequency=orbital_freq, 
                    spin_frequency=-orbital_freq,
                    max_tidal_order_l=max_order_l, eccentricity_truncation_lvl=eccentricity_trunc,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

            tidal_heating_targ_neg = dissipation_results['tidal_heating']
            tidal_torque_neg = dissipation_results['tidal_torque']
            dspin_dt_targ_neg = dissipation_results['spin_rate_derivative']
            de_dt_neg = dissipation_results['eccentricity_derivative']
            da_dt_neg = dissipation_results['semi_major_axis_derivative']
        
            # Heating Manipulation
            max_heating = max(max_heating, np.max(tidal_heating_targ_neg), np.max(tidal_heating_targ_pos))
            min_heating = min(min_heating, np.min(tidal_heating_targ_neg), np.min(tidal_heating_targ_pos))
            
            # Torque Manipulation
            max_torque = max(max_torque, np.max(tidal_torque_neg), np.max(tidal_torque_pos))
            min_torque = min(min_torque, np.min(tidal_torque_neg), np.min(tidal_torque_pos))
            torque_s0_pos, torque_s0_neg = neg_array_for_log_plot(tidal_torque_pos)
            torque_s1_pos, torque_s1_neg = neg_array_for_log_plot(tidal_torque_neg)
        else:
            # Heating Manipulation
            max_heating = max(max_heating, np.max(tidal_heating_targ_pos))
            min_heating = min(min_heating, np.min(tidal_heating_targ_pos))
            
            # Torque Manipulation
            max_torque = max(max_torque, np.max(tidal_torque_pos))
            min_torque = min(min_torque, np.min(tidal_torque_pos))
            torque_s0_pos, torque_s0_neg = neg_array_for_log_plot(tidal_torque_pos)
            
        # Print Heating Difference
        print('Order-l:', max_order_l, 'Truncation:', eccentricity_trunc, 'MaxtoMin Ratio:',
              np.max(tidal_heating_targ_pos)/np.min(tidal_heating_targ_pos))
        
        # Plot results
        style = case_line_styles[case_i]
        name = case_names[case_i]
        
        if double_plot:
            ax0.plot(obliquity_domain_deg, tidal_heating_targ_pos, ls=style, c='k', label=name)
            ax1.plot(obliquity_domain_deg, tidal_heating_targ_neg, ls=style, c='k', label=name)

            ax2_0.plot(obliquity_domain_deg, torque_s0_pos, ls=style, c=red, label=name)
            ax2_0.plot(obliquity_domain_deg, torque_s0_neg, ls=style, c=blue)
            ax2_1.plot(obliquity_domain_deg, torque_s1_pos, ls=style, c=red, label=name)
            ax2_1.plot(obliquity_domain_deg, torque_s1_neg, ls=style, c=blue)
        else:
            ax0.plot(obliquity_domain_deg, tidal_heating_targ_pos, ls=style, c='k', label=name)
            ax2_0.plot(obliquity_domain_deg, torque_s0_pos, ls=style, c=red, label=name)
            ax2_0.plot(obliquity_domain_deg, torque_s0_neg, ls=style, c=blue)
        
    # Legend
    ax0.legend(loc='best')
    
    # Avoid color legend for torque plot. Use heating plot for handles
    handles, labels = ax0.get_legend_handles_labels()
    ax2_0.legend(handles, labels, loc='lower center')
    
    # Axis limits and grid
    for ax in axes:
#         if min_heating < 1e16:
#             min_heating = 0.
#         else:
#             min_heating = 0.99 * min_heating
#         ax.set(ylim=(min_heating, max_heating*1.01))
        ax.set(ylim=(0.2e17, 1.4e17))
        ax.set_xticks([0, 45, 90, 135, 180, 225, 270, 315, 360])
        ax.grid(axis='both', alpha=0.5, ls='--')
        
    for ax in axes2:
        min_torque = max(min_torque, 1e14)
        ax.set(ylim=(min_torque*0.99, max_torque*1.01), yscale='log')
        ax.set_xticks([0, 45,90, 135, 180, 225, 270, 315, 360])
        ax.grid(axis='both', alpha=0.5, ls='--')
    
    if save_fig:
        if different_e:
            save_name = analysis_version_save + 'ObliquityEccenLevels_' + target_object['name'] + '_' + host_object['name']
        else:
            save_name = analysis_version_save + 'ObliquityEccenTruncs_' + target_object['name'] + '_' + host_object['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        fig.savefig(save_name + '_Heating.pdf')
        fig2.savefig(save_name + '_Torque.pdf')
        
    plt.show()

#### Modest Eccentricity, 0.05

In [50]:
obliquity_variation_eccentricity_truncation_1(TRAPPIST1_e, TRAPPIST1, eccentricity=0.05,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper',
                                            double_plot=False,
                                            add_save_name='ModestEccen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Order-l: 2 Truncation: 2 MaxtoMin Ratio: 79.69477473669741
Order-l: 2 Truncation: 6 MaxtoMin Ratio: 78.6908252720574
Order-l: 2 Truncation: 10 MaxtoMin Ratio: 78.69076805589395
Order-l: 2 Truncation: 20 MaxtoMin Ratio: 78.69076805435344


#### Large Eccentricity, 0.3

In [49]:
obliquity_variation_eccentricity_truncation_1(TRAPPIST1_e, TRAPPIST1, eccentricity=0.3,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper',
                                            double_plot=False,
                                            add_save_name='LargeEccen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Order-l: 2 Truncation: 2 MaxtoMin Ratio: 4.043384082915312
Order-l: 2 Truncation: 6 MaxtoMin Ratio: 3.0854311308374442
Order-l: 2 Truncation: 10 MaxtoMin Ratio: 3.0392308721757035
Order-l: 2 Truncation: 20 MaxtoMin Ratio: 3.037590605148129


In [48]:
obliquity_variation_eccentricity_truncation_1(TRAPPIST1_e, TRAPPIST1,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper',
                                            double_plot=False,
                                            add_save_name='LargeEccen', different_e=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Order-l: 2 Truncation: 20 MaxtoMin Ratio: inf
Order-l: 2 Truncation: 20 MaxtoMin Ratio: 20.320788105764322
Order-l: 2 Truncation: 20 MaxtoMin Ratio: 3.037590605148129
Order-l: 2 Truncation: 20 MaxtoMin Ratio: 1.399734907565173


#### Very Large Eccentricity, 0.6

In [47]:
obliquity_variation_eccentricity_truncation_1(TRAPPIST1_e, TRAPPIST1, eccentricity=0.6,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper',
                                            double_plot=False,
                                            add_save_name='VeryLargeEccen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Order-l: 2 Truncation: 2 MaxtoMin Ratio: 2.499534957873485
Order-l: 2 Truncation: 6 MaxtoMin Ratio: 1.6614933731933588
Order-l: 2 Truncation: 10 MaxtoMin Ratio: 1.482022749596828
Order-l: 2 Truncation: 20 MaxtoMin Ratio: 1.399734907565173


### Obliquity Domain - Eccentricity Truncation

In [33]:
def obliquity_variation_eccentricity_truncation_2(target_object, host_object,
                        target_viscosity=5.0e14, target_shear=3.3e9, target_rheo='Sundberg-Cooper',
                        double_plot=True, show_annotation: bool = False,
                        eccentricity=.3, add_save_name: str = None, save_fig=auto_save_plots_ck.value):
    
    # Find rheology information
    rheo_color, rheo_model_name, rheo_input = rheologies[target_rheo]

    # Setup Domain and Cases
    cases = [(2, 2), (2, 6), (2, 10), (2, 20)]
    case_line_styles = ['-', '--', '-.', ':']
    case_names = ['$\\mathcal{O}(e^{' + f'{trunc_level}' + '}$)' if order_l == 2 
                  else '$l = ' + str(order_l) + '\\mathcal{O}(e^{' + f'{trunc_level}' + '}$)'
                  for order_l, trunc_level in cases]
    
    oned_n, twod_n = get_resolution()
    obliquity_domain_deg = np.linspace(0, 360., oned_n)
    obliquity_domain = np.radians(obliquity_domain_deg)
    eccentricity *= np.ones_like(obliquity_domain)
    
    
    spin_rate = target_object['orbital_freq']
    orbital_freq = target_object['orbital_freq']
    
    spin_rate *= np.ones_like(obliquity_domain)
    orbital_freq *= np.ones_like(obliquity_domain)
    semi_major_axis = orbital_motion2semi_a(orbital_freq, host_object['mass'], target_object['mass'])
    
    # Setup plot
    if double_plot:
        fig, axes = plt.subplots(ncols=2, figsize=(6.4*1.75, 4.8), constrained_layout=True)
        ax0 = axes[0]
        ax1 = axes[1]
        ax0.set_title('$\\dot{\\theta} \\; = \\; n$')
        ax1.set_title('$\\dot{\\theta} \\; = \\; -n$')
    else:
        fig, ax0 = plt.subplots(ncols=1, figsize=(6.4, 4.8), constrained_layout=True)
        ax0.set_title('$\\dot{\\theta} \\; = \\; n$')
        ax1 = None
        axes = [ax0]
    
    for ax in fig.get_axes():
        ax.label_outer()
    
    for ax in axes:
        if ax is ax0:
            ax.set(xlabel='Obliquity [deg.]', ylabel='Tidal Heating [W]')
        else:
            ax.set(xlabel='Obliquity [deg.]')
    
    max_heating = 0.
    min_heating = 1.e100
    first_case_heating = None
    last_case_heating = None
    for case_i, (max_order_l, eccentricity_trunc) in enumerate(cases):

        # Calculate tides: spin=n
        dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=eccentricity, obliquity=obliquity_domain,
                    orbital_frequency=orbital_freq,
                    spin_frequency=-orbital_freq,
                    max_tidal_order_l=max_order_l, eccentricity_truncation_lvl=eccentricity_trunc,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

        tidal_heating_targ_pos = dissipation_results['tidal_heating']
        tidal_torque_pos = dissipation_results['tidal_torque']
        dspin_dt_targ_pos = dissipation_results['spin_rate_derivative']
        de_dt_pos = dissipation_results['eccentricity_derivative']
        da_dt_pos = dissipation_results['semi_major_axis_derivative']

        if ax1 is not None:
            dissipation_results = \
                single_dissipation_from_dict_or_world_instance(
                    host_object, target_object,
                    target_viscosity, target_shear, rheology=rheo_model_name,
                    complex_compliance_inputs=rheo_input,
                    eccentricity=eccentricity, obliquity=obliquity_domain,
                    orbital_frequency=orbital_freq,
                    spin_frequency=-orbital_freq,
                    max_tidal_order_l=max_order_l, eccentricity_truncation_lvl=eccentricity_trunc,
                    use_obliquity=True, tidal_scale=target_object['tidal_scale'],
                    fixed_k2=target_object['k2'], fixed_q=target_object['fixed_q'],
                    fixed_dt=target_object['fixed_dt'],
                    da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                    dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))

            tidal_heating_targ_neg = dissipation_results['tidal_heating']
            tidal_torque_neg = dissipation_results['tidal_torque']
            dspin_dt_targ_neg = dissipation_results['spin_rate_derivative']
            de_dt_neg = dissipation_results['eccentricity_derivative']
            da_dt_neg = dissipation_results['semi_major_axis_derivative']

        
            max_heating = max(max_heating, np.max(tidal_heating_targ_pos))
            min_heating = min(min_heating, np.min(tidal_heating_targ_pos))
        else:
            max_heating = max(max_heating, np.max(tidal_heating_targ_pos))
            min_heating = min(min_heating, np.min(tidal_heating_targ_pos))
        
        # Plot results
        style = case_line_styles[case_i]
        name = case_names[case_i]
        ax0.plot(obliquity_domain_deg, tidal_heating_targ_pos, ls=style, c='k', label=name)
        if ax1 is not None:
            ax1.plot(obliquity_domain_deg, tidal_heating_targ_neg, ls=style, c='k', label=name)
            
        if case_i == 0:
            first_case_heating = tidal_heating_targ_pos
        elif case_i == len(cases) - 1:
            last_case_heating = tidal_heating_targ_pos
            
        # Plot Enhancement Line
        if case_i == len(cases) - 1:
            no_obliquity_heating = tidal_heating_targ_pos[0]
            max_obliquity_heating = np.max(tidal_heating_targ_pos)
            max_obliquity_heating_loc = find_nearest(tidal_heating_targ_pos, max_obliquity_heating)
            max_obliq_value = np.degrees(obliquity_domain[max_obliquity_heating_loc])
            print(max_obliq_value)
            heating_delta = max_obliquity_heating-no_obliquity_heating
            
            # Add hline
            ax0.axhline(y=no_obliquity_heating, ls=':', c='b', alpha=1.)
            
            # Add Arrows
            if show_annotation:
                ax0.annotate("", xy=(max_obliq_value, no_obliquity_heating),
                             xytext=(max_obliq_value, max_obliquity_heating),
                             xycoords='data', textcoords='data', arrowprops={'arrowstyle': '<->', 'color': 'blue'})

                # Add Text
                increase_text = "Maximum heating enhancement of \n$\\approx " + f"{max_obliquity_heating/no_obliquity_heating:.0f}$" + "$\\times$ from obliquity tides."

                ax0.text(max_obliq_value, 0.7 * no_obliquity_heating, increase_text,
                         fontsize=12, ha='center', c='b')
        
    ax0.legend(loc='lower right')
    for ax in axes:
        ax.set(ylim=(min_heating*0.7, max_heating*1.05))
        ax.set_xticks([0, 45,90, 135, 180, 225, 270, 315, 360])
        ax.grid(axis='both', alpha=0.5, ls='--')
    
    # Make Ratio Plot
    if False:
        fig2, ax2_0 = plt.subplots(ncols=1, figsize=(6.4, 4.8), constrained_layout=True)
        ax2_0.plot(obliquity_domain_deg, last_case_heating/first_case_heating)
        ax2_0.set(ylabel=f'Tidal Heating Ratio {case_names[len(cases) - 1]} / {case_names[0]}', xlabel='Obliquity [deg.]')
    
    if save_fig:
        save_name = analysis_version_save + 'ObliquityEccenTruncs_' + target_object['name'] + '_' + host_object['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        fig.savefig(save_name + '_ObliquityFig.pdf')
        

#### Modest Eccentricity, 0.05

In [46]:
obliquity_variation_eccentricity_truncation_2(TRAPPIST1_e, TRAPPIST1, eccentricity=0.05,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper', double_plot=False,
                                            add_save_name='ModestEccen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

325.0433008660173


#### Large Eccentricity, 0.3

In [45]:
obliquity_variation_eccentricity_truncation_2(TRAPPIST1_e, TRAPPIST1, eccentricity=0.3,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper', double_plot=False,
                                            add_save_name='LargeEccen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

26.28772575451509


#### Very Large Eccentricity, 0.6

In [44]:
obliquity_variation_eccentricity_truncation_2(TRAPPIST1_e, TRAPPIST1, eccentricity=0.6,
                                            target_viscosity=rock_viscosity, target_shear=rock_shear,
                                            target_rheo='Sundberg-Cooper', double_plot=False,
                                            add_save_name='VeryLargeEccen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0


## Pluto-Charon de/dt over semi-major axis domain

## Higher Order l Comparison

In [37]:
def higher_order_l_compare(object1_dict, object2_dict,
                           obj1_viscosity=ice_viscosity, obj1_shear=ice_shear, obj1_rheology='Sundberg-Cooper',
                           obj2_viscosity=ice_viscosity, obj2_shear=ice_shear, obj2_rheology='Sundberg-Cooper',
                           eccentricity_truncation=10,
                           eccentricities=(0.01, 0.6), obj1_obliquity = 0., obj2_obliquity = 0.,
                           zpoints_heating=np.linspace(1, 2., 30), zticks_heating=(1, 1.25, 1.5, 1.75, 2),
                           zpoints_eccen=np.linspace(-2, 2., 30), zticks_eccen=(-2, -1, 0, 1, 2),
                           ztick_names = ('$10^{-2}$', '$10^{-1}$', '$1$', '$10^{1}$', '$10^{2}$'),
#                            zpoints_eccen=50, zticks_eccen=None,
                           add_save_name: str = None, save_fig=auto_save_plots_ck.value):
    
    print('This plot can take a while even at low resolution.')
    order_ls = [2, 3, 7]
    obj1_tidal_calcs = dict()
    obj2_tidal_calcs = dict()
    eccentricity_calcs = dict()
    
    # Pull out information
    rheo1_color, rheo1_model_name, rheo1_input = rheologies[obj1_rheology]
    rheo2_color, rheo2_model_name, rheo2_input = rheologies[obj2_rheology]
    
    # Setup Domains
    oned_n, twod_n = get_resolution()
    # This plot can use a ton of memory so its N should be lower than all the other plots.
    twod_n_real = int(math.floor(0.55 * twod_n))
    orbital_period_domain = np.logspace(-1, 1, twod_n_real)
    obj1_spin_period_domain =  np.logspace(-1, 1, twod_n_real)
    
    orbital_freq_domain = days2rads(orbital_period_domain)
    obj1_spin_freq_domain = days2rads(obj1_spin_period_domain)
    
    orbital_freq, obj1_spin_freq = np.meshgrid(orbital_freq_domain, obj1_spin_freq_domain)
    shape = orbital_freq.shape
    orbital_freq = orbital_freq.flatten()
    obj1_spin_freq = obj1_spin_freq.flatten()
    semi_major_axis = orbital_motion2semi_a(orbital_freq, object1_dict['mass'], object2_dict['mass'])
    x = orbital_period_domain
    y = obj1_spin_period_domain
    
    #    Assume object 2 has already spin-synched
    obj2_spin_freq = orbital_freq # 2. * np.pi / (86400. * 2.) * np.ones_like(orbital_freq)
    
    # Setup figures
    #    Heating Figure
    heating_fig = plt.figure(figsize=(6.4*1.5, 6.8*1.5), constrained_layout=True)
    ratios = (.49, .49, .02)
    heating_gs = GridSpec(3, 3, figure=heating_fig, width_ratios=ratios)
    heating_axes = [heating_fig.add_subplot(heating_gs[0, i]) for i in range(2)]
    heating_axes += [heating_fig.add_subplot(heating_gs[1, i]) for i in range(2)]
    heating_axes += [heating_fig.add_subplot(heating_gs[2, i]) for i in range(2)]
    heating_cbs = [heating_fig.add_subplot(heating_gs[i, 2]) for i in range(3)]
    
    for ax_i, ax in enumerate(heating_axes):
        ax.set(xscale='log', yscale='log')
        if ax_i == 0:
            ax.set(ylabel='$l_{max}=2$\nSpin Period [days]', title=f'e={eccentricities[0]}')
            ax.tick_params(labelbottom=False)
        elif ax_i == 1:
            ax.set(title=f'e={eccentricities[1]}')
            ax.tick_params(labelbottom=False, labelleft=False)
        elif ax_i == 2:
            ax.set(ylabel='$l_{max}=3$\nSpin Period [days]')
            ax.tick_params(labelbottom=False)
        elif ax_i == 3:
            ax.tick_params(labelbottom=False, labelleft=False)
        elif ax_i == 4:
            ax.set(ylabel='$l_{max}=7$\nSpin Period [days]', xlabel='Orbital Period [days]')
        elif ax_i == 5:
            ax.set(xlabel='Orbital Period [days]')            
            ax.tick_params(labelleft=False)
            
    #    Eccentricity Figure
    eccen_fig = plt.figure(figsize=(6.4*1.5, 6.8*1.5), constrained_layout=True)
    ratios = (.49, .49, .02)
    eccen_gs = GridSpec(3, 3, figure=eccen_fig, width_ratios=ratios)
    eccen_axes = [eccen_fig.add_subplot(eccen_gs[0, i]) for i in range(2)]
    eccen_axes += [eccen_fig.add_subplot(eccen_gs[1, i]) for i in range(2)]
    eccen_axes += [eccen_fig.add_subplot(eccen_gs[2, i]) for i in range(2)]
    eccen_cbs = [eccen_fig.add_subplot(eccen_gs[i, 2]) for i in range(3)]
    
    for ax_i, ax in enumerate(eccen_axes):
        ax.set(xscale='log', yscale='log')
        if ax_i == 0:
            ax.set(ylabel='$l_{max}=2$\nSpin Period [days]', title=f'e={eccentricities[0]}')
            ax.tick_params(labelbottom=False)
        elif ax_i == 1:
            ax.set(title=f'e={eccentricities[1]}')
            ax.tick_params(labelbottom=False, labelleft=False)
        elif ax_i == 2:
            ax.set(ylabel='$l_{max}=3$\nSpin Period [days]')
            ax.tick_params(labelbottom=False)
        elif ax_i == 3:
            ax.tick_params(labelbottom=False, labelleft=False)
        elif ax_i == 4:
            ax.set(ylabel='$l_{max}=7$\nSpin Period [days]', xlabel='Orbital Period [days]')
        elif ax_i == 5:
            ax.set(xlabel='Orbital Period [days]')            
            ax.tick_params(labelleft=False)
        
    
    case_inputs = {
        (3, eccentricities[0], 0): (heating_axes[2], eccen_axes[2]),
        (3, eccentricities[1], 1): (heating_axes[3], eccen_axes[3]),
        (7, eccentricities[0], 0): (heating_axes[4], eccen_axes[4]),
        (7, eccentricities[1], 1): (heating_axes[5], eccen_axes[5]),
    }
    
    # Calculate baseline (order-l=2)
    obj1_heatings_2 = list()
    dedts_2 = list()
    for e_i, eccentricity in enumerate(eccentricities):
        
        # Calculate Dissipation and derivatives
        dissipation_results = \
            dual_dissipation_from_dict_or_world_instance(
                object1_dict, object2_dict,
                viscosities=(obj1_viscosity, obj2_viscosity),
                shear_moduli=(obj1_shear, obj2_shear),
                rheologies=(rheo1_model_name, rheo2_model_name),
                complex_compliance_inputs=(rheo1_input, rheo2_input),
                obliquities=(obj1_obliquity, obj2_obliquity),
                spin_frequencies=(obj1_spin_freq, obj2_spin_freq),
                tidal_scales=(object1_dict['tidal_scale'], object2_dict['tidal_scale']),
                fixed_k2s=(object1_dict['k2'], object2_dict['k2']),
                fixed_qs=(object1_dict['fixed_q'], object2_dict['fixed_q']),
                fixed_dts=(object1_dict['fixed_dt'], object2_dict['fixed_dt']),
                eccentricity=eccentricity,
                orbital_frequency=orbital_freq,
                max_tidal_order_l=2, eccentricity_truncation_lvl=eccentricity_truncation,
                use_obliquity=True,
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))
        
        obj1_tidal_heating_2 = dissipation_results['host']['tidal_heating']
        de_dt = dissipation_results['eccentricity_derivative']
        da_dt = dissipation_results['semi_major_axis_derivative']
        
        # Reshape results and store
        obj1_tidal_heating_2 = obj1_tidal_heating_2.reshape(shape)
        de_dt = de_dt.reshape(shape)
        obj1_heatings_2.append(obj1_tidal_heating_2)
        dedts_2.append(de_dt)
        
        # l=2 data for plots
        heating_data = np.log10(obj1_tidal_heating_2)
        eccen_data_combo = np.log10(np.abs(de_dt))
        
        # Add to plots
        heating_ax = heating_axes[e_i]
        eccen_ax = eccen_axes[e_i]
        
        heating_cb_data_top = heating_ax.contourf(x, y, heating_data, 60, cmap=lajolla_map)        
        eccen_cb_data_top = eccen_ax.contourf(x, y, eccen_data_combo, np.linspace(-8, 3, len(zpoints_eccen)), cmap=lajolla_map)
        for c in eccen_cb_data_top.collections:
            c.set_edgecolor("face")
        for c in heating_cb_data_top.collections:
            c.set_edgecolor("face")
        
        
    # Perform other l-order calculations and add to the plot
    eccen_cb_datas = list()
    for (order_l, eccentricity, order_2_index), (heating_ax, eccen_ax) in case_inputs.items():
        
        # Calculate Dissipation and derivatives
        dissipation_results = \
            dual_dissipation_from_dict_or_world_instance(
                object1_dict, object2_dict,
                viscosities=(obj1_viscosity, obj2_viscosity),
                shear_moduli=(obj1_shear, obj2_shear),
                rheologies=(rheo1_model_name, rheo2_model_name),
                complex_compliance_inputs=(rheo1_input, rheo2_input),
                obliquities=(obj1_obliquity, obj2_obliquity),
                spin_frequencies=(obj1_spin_freq, obj2_spin_freq),
                tidal_scales=(object1_dict['tidal_scale'], object2_dict['tidal_scale']),
                fixed_k2s=(object1_dict['k2'], object2_dict['k2']),
                fixed_qs=(object1_dict['fixed_q'], object2_dict['fixed_q']),
                fixed_dts=(object1_dict['fixed_dt'], object2_dict['fixed_dt']),
                eccentricity=eccentricity,
                orbital_frequency=orbital_freq,
                max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                use_obliquity=True,
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))
    
        obj1_tidal_heating = dissipation_results['host']['tidal_heating']
        de_dt = dissipation_results['eccentricity_derivative']
        da_dt = dissipation_results['semi_major_axis_derivative']
        
        # Reshape
        obj1_tidal_heating = obj1_tidal_heating.reshape(shape)
        de_dt = de_dt.reshape(shape)
        
        # Pull out order-2 results and find ratio
        heating_ratio = np.abs(obj1_tidal_heating / obj1_heatings_2[order_2_index])
        eccen_ratio = np.abs(de_dt / dedts_2[order_2_index])
        eccen_ratio = np.log10(eccen_ratio)
        
        ## There are a handful of points above 10^2 for l=7 at very low spin-period. Set this to 10^2 just so the plot is filled out.
        eccen_ratio[eccen_ratio > 2] = 2
        
        # Add to plots
        heating_cb_data = heating_ax.contourf(x, y, heating_ratio, zpoints_heating, cmap=lajolla_map)
        
        eccen_cb_data = eccen_ax.contourf(x, y, eccen_ratio, zpoints_eccen, cmap=cork_map)
        eccen_cb_datas.append(eccen_cb_data)
        
        for c in eccen_cb_data.collections:
            c.set_edgecolor("face")
        for c in heating_cb_data.collections:
            c.set_edgecolor("face")
        
    # Add Colorbars
    for cb_i, cb_ax in enumerate(heating_cbs):
        if cb_i == 0:
            cb = plt.colorbar(heating_cb_data_top, pad=0.05, cax=cb_ax)
            cb.set_label('Tidal Heating [log$_{10}$ W]')
        else:
            cb = plt.colorbar(heating_cb_data, pad=0.05, cax=cb_ax, ticks=zticks_heating)
            cb.set_label('Tidal Heating Ratio')
            
    # Annotations
    eccen_axes[3].annotate("Enhancement at\nLow Orbital Period", (0.4, 4), (0.4, 4),
                           fontsize=12, ha='center')
    eccen_axes[5].annotate("New Modes\nActivated", (0.4, .29), (0.26, 1),
                           arrowprops={'arrowstyle': '-['},
                           fontsize=12, ha='center')
    # Add Colorbars
    for cb_i, cb_ax in enumerate(heating_cbs):
        if cb_i == 0:
            cb = plt.colorbar(heating_cb_data_top, pad=0.05, cax=cb_ax)
            cb.set_label('Tidal Heating [log$_{10}$ W]')
        else:
            cb = plt.colorbar(heating_cb_data, pad=0.05, cax=cb_ax, ticks=zticks_heating)
            cb.set_label('Tidal Heating Ratio')
            
    for cb_i, cb_ax in enumerate(eccen_cbs):
        if cb_i == 0:
            cb = plt.colorbar(eccen_cb_data_top, pad=0.05, cax=cb_ax, ticks=[-8, -6, -4, -2, 0, 2])
            cb.set_label('$|\\dot{e}|$ [yr$^{-1}$]')
            cb.ax.set_yticklabels(['$10^{-8}$', '$10^{-6}$', '$10^{-4}$', '$10^{-2}$', '$1$', '$10^{2}$'])
        else:
            cb2 = plt.colorbar(eccen_cb_datas[cb_i-1], pad=0.05, cax=eccen_cbs[cb_i], ticks=zticks_eccen)
            cb2.set_label('$|\\dot{e}|$ Ratio')
            cb2.ax.set_yticklabels(ztick_names)
        
    if save_fig:
        save_name = analysis_version_save + 'OrderlComparePlot_' + object1_dict['name'] + '_' + object2_dict['name']
        if add_save_name is not None:
            save_name = save_name + '_' + add_save_name

        heating_fig.savefig(save_name + '_HeatingRatio.pdf')
        eccen_fig.savefig(save_name + '_dedtRatio.pdf')
        heating_fig.savefig(save_name + '_HeatingRatio.png', dpi=300)
        eccen_fig.savefig(save_name + '_dedtRatio.png', dpi=300)

    plt.show()

In [43]:
higher_order_l_compare(pluto, charon,
                       obj1_viscosity=ice_viscosity, obj1_shear=ice_shear, obj1_rheology='Sundberg-Cooper',
                           obj2_viscosity=ice_viscosity, obj2_shear=ice_shear, obj2_rheology='Sundberg-Cooper',
                           eccentricity_truncation=20,
                           eccentricities=(0.05, 0.6), obj1_obliquity = 0., obj2_obliquity = 0., add_save_name='IceVisco')

This plot can take a while even at low resolution.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
higher_order_l_compare(pluto, charon,
                       obj1_viscosity=ice_viscosity, obj1_shear=ice_shear, obj1_rheology='CTL',
                           obj2_viscosity=ice_viscosity, obj2_shear=ice_shear, obj2_rheology='CTL',
                           eccentricity_truncation=20,
                           eccentricities=(0.05, 0.6), obj1_obliquity = 0., obj2_obliquity = 0., add_save_name='IceVisco')

This plot can take a while even at low resolution.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Interactive

### Order-l Comparison: Eccentricity leads to resonance shifts

In [40]:
%matplotlib notebook

def resonance_shift_interactive(object1_dict, object2_dict,
                                obj1_viscosity=ice_viscosity, obj1_shear=ice_shear, obj1_rheology='Sundberg-Cooper',
                                obj2_viscosity=ice_viscosity, obj2_shear=ice_shear, obj2_rheology='Sundberg-Cooper',
                                eccentricity_truncation=20,
                                obj1_obliquity = 0., obj2_obliquity = 0.,
                                order_l: int = 3, computational_N = None,
                                save_frame: bool = False,
                                add_save_name: str = None, save_fig=auto_save_plots_ck.value):
    
    # Interpret input and reinit function
    rheo1_color, rheo1_model_name, rheo1_input = rheologies[obj1_rheology]
    rheo2_color, rheo2_model_name, rheo2_input = rheologies[obj2_rheology]
    
    # Setup domains
    if computational_N is None:
        oned_n, twod_n = get_resolution()
        computational_N = oned_n
        
    orbital_period = np.linspace(.5, 40., computational_N)
    orbital_freq = days2rads(orbital_period)
    semi_major_axis = orbital_motion2semi_a(orbital_freq, object1_dict['mass'], object2_dict['mass'])
    x = orbital_period
    
    # Setup plot
    fig, ax = plt.subplots()
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.resizable = True
    ax.set(xlabel='Orbital Period [days]', ylabel='$\\dot{e}$ Ratio [$l=' + f'{order_l}' + '/l=2$]', ylim=(0, 10))
    # Initialize plot with fake data
    line_pos, = ax.plot(x, np.ones_like(x), c=red)
    line_neg, = ax.plot(x, np.ones_like(x), c=blue)
    obj1_vline = ax.axvline(x=1., ls=':', c=red, alpha=0.5)
    obj2_vline = ax.axvline(x=2., ls=':', c=blue, alpha=0.5)
    
    # Make a directory for frames
    if save_frame and save_fig:
        dir_name = 'eccen_resonance_frames'
        if add_save_name is not None:
            dir_name += add_save_name
        frame_dir = os.path.join(os.getcwd(), dir_name)
        if not os.path.isdir(frame_dir):
            os.mkdir(frame_dir)
    
    def resonance_shift(object_1_spin_period, object_2_spin_period, eccentricity):
        
        # Find frequencies
        object1_spin_freq = days2rads(object_1_spin_period)
        object2_spin_freq = days2rads(object_2_spin_period)
        
        # Calculate l=2 Dissipation
        dissipation_results = \
            dual_dissipation_from_dict_or_world_instance(
                object1_dict, object2_dict,
                viscosities=(obj1_viscosity, obj2_viscosity),
                shear_moduli=(obj1_shear, obj2_shear),
                rheologies=(rheo1_model_name, rheo2_model_name),
                complex_compliance_inputs=(rheo1_input, rheo2_input),
                obliquities=(obj1_obliquity, obj2_obliquity),
                spin_frequencies=(object1_spin_freq, object2_spin_freq),
                tidal_scales=(object1_dict['tidal_scale'], object2_dict['tidal_scale']),
                fixed_k2s=(object1_dict['k2'], object2_dict['k2']),
                fixed_qs=(object1_dict['fixed_q'], object2_dict['fixed_q']),
                fixed_dts=(object1_dict['fixed_dt'], object2_dict['fixed_dt']),
                eccentricity=eccentricity,
                orbital_frequency=orbital_freq,
                max_tidal_order_l=2, eccentricity_truncation_lvl=eccentricity_truncation,
                use_obliquity=True,
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))
    
        obj1_tidal_heating_2 = dissipation_results['host']['tidal_heating']
        obj2_tidal_heating_2 = dissipation_results['secondary']['tidal_heating']
        dspin_dt_obj1_2 = dissipation_results['host']['spin_rate_derivative']
        dspin_dt_obj2_2 = dissipation_results['secondary']['spin_rate_derivative']
        de_dt_2 = dissipation_results['eccentricity_derivative']
        da_dt_2 = dissipation_results['semi_major_axis_derivative']
        
        # Calculate provided l Dissipation
        dissipation_results = \
            dual_dissipation_from_dict_or_world_instance(
                object1_dict, object2_dict,
                viscosities=(obj1_viscosity, obj2_viscosity),
                shear_moduli=(obj1_shear, obj2_shear),
                rheologies=(rheo1_model_name, rheo2_model_name),
                complex_compliance_inputs=(rheo1_input, rheo2_input),
                obliquities=(obj1_obliquity, obj2_obliquity),
                spin_frequencies=(object1_spin_freq, object2_spin_freq),
                tidal_scales=(object1_dict['tidal_scale'], object2_dict['tidal_scale']),
                fixed_k2s=(object1_dict['k2'], object2_dict['k2']),
                fixed_qs=(object1_dict['fixed_q'], object2_dict['fixed_q']),
                fixed_dts=(object1_dict['fixed_dt'], object2_dict['fixed_dt']),
                eccentricity=eccentricity,
                orbital_frequency=orbital_freq,
                max_tidal_order_l=order_l, eccentricity_truncation_lvl=eccentricity_truncation,
                use_obliquity=True,
                da_dt_scale=(3.154e7 / 1000.), de_dt_scale=3.154e7,
                dspin_dt_scale=((360. / (2. * np.pi)) * 3.154e7**2))
    
        obj1_tidal_heating_l = dissipation_results['host']['tidal_heating']
        obj2_tidal_heating_l = dissipation_results['secondary']['tidal_heating']
        dspin_dt_obj1_l = dissipation_results['host']['spin_rate_derivative']
        dspin_dt_obj2_l = dissipation_results['secondary']['spin_rate_derivative']
        de_dt_l = dissipation_results['eccentricity_derivative']
        da_dt_l = dissipation_results['semi_major_axis_derivative']
        
        # Calculate Ratios
        eccen_ratio = de_dt_l / de_dt_2
        # Make eccentriciy work with log plotting (likely to have negatives)
        eccen_ratio_pos, eccen_ratio_neg = neg_array_for_log_plot(eccen_ratio)
        
        return eccen_ratio_pos, eccen_ratio_neg
    
    def update(object_1_spin_period=2., object_2_spin_period=4., eccentricity=.3):
        
        # Calculate new ratios
        eccen_ratio_pos, eccen_ratio_neg = resonance_shift(object_1_spin_period, object_2_spin_period, eccentricity)
        
        # Update plot
        line_pos.set_ydata(eccen_ratio_pos)
        line_neg.set_ydata(eccen_ratio_neg)
        ax.set(title=f'e={eccentricity:0.2f}, s1={object_1_spin_period:.1f}, s2={object_2_spin_period:0.1f}')
        obj1_vline.set_xdata((object_1_spin_period, object_1_spin_period))
        obj2_vline.set_xdata((object_2_spin_period, object_2_spin_period))
        
        if save_frame and save_fig:
            frame_name = f'eccen_resonance_frame__sp1_{object_1_spin_period:.1f}__sp2' + \
                         f'_{object_2_spin_period:.1f}__e_{eccentricity:0.2f}.png'
            fig.savefig(os.path.join(frame_dir, frame_name), dpi=300)
        
        # Recalculate the plot limits
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()
        
    interact(update,
         object_1_spin_period=(0.5,10,.25),
         object_2_spin_period=(0.5,10,.25),
         eccentricity=(0.001, 0.7, 0.05))
    

In [41]:
breakpoint()

--Call--
> c:\users\joepr\.conda\envs\dev\lib\site-packages\ipython\core\displayhook.py(252)__call__()
    250         sys.stdout.flush()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.
    254 

ipdb> q


BdbQuit: 

In [ ]:
resonance_shift_interactive(pluto, charon,
                            obj1_viscosity=ice_viscosity, obj1_shear=ice_shear, obj1_rheology='Sundberg-Cooper',
                            obj2_viscosity=ice_viscosity, obj2_shear=ice_shear, obj2_rheology='Sundberg-Cooper',
                            eccentricity_truncation=14,
                            obj1_obliquity=0., obj2_obliquity=0.,
                            order_l=4, computational_N=2000,
                            save_frame=False, add_save_name=None, save_fig=False)